In [ ]:
 !pip install rioxarray "xarray[complete]" geopandas  "dask[complete]"

In [1]:
import rioxarray as rx
import pandas as pd
import boto3
import xarray as xr
import glob
import shutil
import os
import zarr
import s3fs
import geopandas as gpd
from dask.diagnostics import ProgressBar

In [2]:
pathS3 = "s3://climate-action-datalake/zone=raw/source=agera5"
localPath = 'data/toZarr/*.nc'

In [3]:
def convertKelvinCelsius(k):
    c = k - 273.15
    return c

In [4]:
allFiles = xr.open_mfdataset(localPath,parallel=True)

In [5]:
print('Hola mundo')

Hola mundo


In [6]:
allFiles = allFiles.apply(convertKelvinCelsius)

In [7]:
compressor = zarr.Blosc(cname='zstd')
encoding = {vname: {'compressor': compressor,'chunks': (1,1801,3600)} for vname in allFiles.data_vars}


In [8]:
s3 = s3fs.S3FileSystem()
store = s3fs.S3Map(root="s3://climate-action-datalake/zone=raw/source=agera5/variable=airTemperatureMax.zarr", s3=s3, check=False)

In [9]:
with ProgressBar():
    result = allFiles.to_zarr(store=store, encoding=encoding, consolidated=True)

[########################################] | 100% Completed | 53m 4ss


In [10]:
print("He finalizado")

He finalizado
